**Tarea World Bank's international - ETL**

**Fecha: 14 de enero de 2021**

**Autor: Diego Senso González**

## 1. The World Bank's international debt data
<p>No es que los humanos solo tengamos deudas para administrar nuestras necesidades. Un país también puede endeudarse para administrar su economía. Por ejemplo, el gasto en infraestructura es un ingrediente costoso requerido para que los ciudadanos de un país lleven una vida cómoda. El Banco Mundial es la organización que proporciona deuda a los países.</p>

<!-- <p>En este notebook, vamos a analizar los datos de la deuda internacional recopilados por el Banco Mundial. El conjunto de datos contiene información sobre el monto de la deuda (en USD) que deben los países en desarrollo en varias categorías.</p>  -->
    

"Disbursements on external debt, long-term (DIS, current US$)",DT.DIS.DLXF.CD
"Interest payments on external debt, long-term (INT, current US$)",DT.INT.DLXF.CD
"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD
"PPG, bilateral (DIS, current US$)",DT.DIS.BLAT.CD
"PPG, bilateral (INT, current US$)",DT.INT.BLAT.CD
"PPG, multilateral (AMT, current US$)",DT.AMT.MLAT.CD
"PPG, multilateral (DIS, current US$)",DT.DIS.MLAT.CD
"PPG, multilateral (INT, current US$)",DT.INT.MLAT.CD
"PPG, official creditors (AMT, current US$)",DT.AMT.OFFT.CD
"PPG, official creditors (DIS, current US$)",DT.DIS.OFFT.CD
"PPG, official creditors (INT, current US$)",DT.INT.OFFT.CD
"Principal repayments on external debt, long-term (AMT, current US$)",DT.AMT.DLXF.CD
"Interest payments on external debt, private nonguaranteed (PNG) (INT, current US$)",DT.INT.DPNG.CD
"PPG, bonds (AMT, current US$)",DT.AMT.PBND.CD
"PPG, bonds (INT, current US$)",DT.INT.PBND.CD
"PPG, commercial banks (AMT, current US$)",DT.AMT.PCBK.CD
"PPG, commercial banks (DIS, current US$)",DT.DIS.PCBK.CD
"PPG, commercial banks (INT, current US$)",DT.INT.PCBK.CD
"PPG, other private creditors (AMT, current US$)",DT.AMT.PROP.CD
"PPG, other private creditors (DIS, current US$)",DT.DIS.PROP.CD
"PPG, other private creditors (INT, current US$)",DT.INT.PROP.CD
"PPG, private creditors (AMT, current US$)",DT.AMT.PRVT.CD
"PPG, private creditors (DIS, current US$)",DT.DIS.PRVT.CD
"PPG, private creditors (INT, current US$)",DT.INT.PRVT.CD
"Principal repayments on external debt, private nonguaranteed (PNG) (AMT, current US$)",DT.AMT.DPNG.CD

<p>Vamos a encontrar las respuestas a preguntas como:

<p>¿Cuál es el monto total de la deuda que deben los países enumerados en el conjunto de datos?
<p>¿Qué país posee la cantidad máxima de deuda y cómo se ve esa cantidad?
<p>¿Cuál es el monto promedio de la deuda de los países a través de diferentes indicadores de deuda?
    
Además tenemos otro dataset en el que encontramos información histórica de algunos índices de desarrollo, entre los que se encuentran algunos de deuda como son:

Series Name,Series Code
"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN

"Central government debt, total (current LCU)",GC.DOD.TOTL.CN

"Central government debt, total (% of GDP)",GC.DOD.TOTL.GD.ZS

#### 1. Inicializar y cargar el contexto spark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz #d
!pip install -q findspark 

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Cargamos los datasets. Los que nos serán de utilidad serán los dos últimos.

In [2]:
var1920 = sc.textFile("explicacion_var1920.csv")
vardeu1920 = sc.textFile("explicacion_varDeu1920.csv")
desarrollo = sc.textFile("indicadores_desarrollo.csv")
deuda = sc.textFile("indicadores_deuda.csv")

Observamos los datos:

In [3]:
desarrollo.take(5)

['country_name,country_code,anho,GC.DOD.TOTL.CN,GC.DOD.TOTL.GD.ZS,SP.DYN.CBRT.IN',
 'Albania,ALB,2014 [YR2014],1023040800000,73.3202469198429,12.259',
 'Albania,ALB,2015 [YR2015],1145500940000,79.8644448747259,12.197',
 'Albania,ALB,2016 [YR2016],1188814490000,80.7355748396923,12.08',
 'Australia,AUS,2014 [YR2014],674700000,0.042207528166503,13.2']

Eliminamos la primera fila, el encabezado:

In [4]:
primera_fila_desarrollo = desarrollo.take(1)[0]
desarrollo = desarrollo.filter(lambda x: x != primera_fila_desarrollo)

Separamos los datos de desarrollo por la coma, y observamos cómo se quedan estructurados los datos. Ya hemos eliminado la primera fila, por lo que ya solo quedan los datos de cada país para diferentes años.

In [5]:
desarrollo_new = desarrollo.map(lambda x: x.split(","))
desarrollo_new.take(5)

[['Albania',
  'ALB',
  '2014 [YR2014]',
  '1023040800000',
  '73.3202469198429',
  '12.259'],
 ['Albania',
  'ALB',
  '2015 [YR2015]',
  '1145500940000',
  '79.8644448747259',
  '12.197'],
 ['Albania',
  'ALB',
  '2016 [YR2016]',
  '1188814490000',
  '80.7355748396923',
  '12.08'],
 ['Australia',
  'AUS',
  '2014 [YR2014]',
  '674700000',
  '0.042207528166503',
  '13.2'],
 ['Australia',
  'AUS',
  '2015 [YR2015]',
  '762718000',
  '0.0469480198522591',
  '12.9']]

Observamos el otro dataset, el de los indicadores de deuda, y realizamos el mismo proceso que para los otros datos.

In [6]:
deuda.take(5)

['country_name;country_code;DT.AMT.BLAT.CD;DT.AMT.DLXF.CD;DT.AMT.DPNG.CD;DT.AMT.MLAT.CD;DT.AMT.OFFT.CD;DT.AMT.PBND.CD;DT.AMT.PCBK.CD;DT.AMT.PROP.CD;DT.AMT.PRVT.CD;DT.DIS.BLAT.CD;DT.DIS.DLXF.CD;DT.DIS.MLAT.CD;DT.DIS.OFFT.CD;DT.DIS.PCBK.CD;DT.DIS.PROP.CD;DT.DIS.PRVT.CD;DT.INT.BLAT.CD;DT.INT.DLXF.CD;DT.INT.DPNG.CD;DT.INT.MLAT.CD;DT.INT.OFFT.CD;DT.INT.PBND.CD;DT.INT.PCBK.CD;DT.INT.PROP.CD;DT.INT.PRVT.CD;',
 'Afghanistan;AFG;61739336.9;100847181.9;;39107845;100847181.9;;;;;49114729.4;72894453.7;23779724.3;72894453.7;;;;39903620.1;53239440.1;;13335820;53239440.1;;;;;',
 'Albania;ALB;54250280.6;790248675.2;514185620;182197616.7;236447897.3;0;39445139.5;170018.4;39615157.9;76050616.1;317194512.5;234321242.3;310371858.4;4542664.9;2279989.2;6822654.1;13847333.6;165602386.9;87884000;28101536.1;41948869.7;31030688.2;4618504.3;120324.7;35769517.2;',
 'Algeria;DZA;95188724.6;171185188.1;75420000;0;95188724.6;;576463.5;0;576463.5;10320772.2;10320772.2;;10320772.2;;;;7680627.6;19031728.7;3220000;80947

Eliminamos el encabezado:

In [7]:
primera_fila_deuda = deuda.take(1)[0]
deuda = deuda.filter(lambda x: x != primera_fila_deuda)

Separamos los campos, esta vez con punto y coma. Observamos que este dataset cuenta con una observación para cada país que recoge numerosos datos relativos a indicadores de deuda.

In [8]:
deuda_new = deuda.map(lambda x: x.split(";"))
deuda_new.take(2)

[['Afghanistan',
  'AFG',
  '61739336.9',
  '100847181.9',
  '',
  '39107845',
  '100847181.9',
  '',
  '',
  '',
  '',
  '49114729.4',
  '72894453.7',
  '23779724.3',
  '72894453.7',
  '',
  '',
  '',
  '39903620.1',
  '53239440.1',
  '',
  '13335820',
  '53239440.1',
  '',
  '',
  '',
  '',
  ''],
 ['Albania',
  'ALB',
  '54250280.6',
  '790248675.2',
  '514185620',
  '182197616.7',
  '236447897.3',
  '0',
  '39445139.5',
  '170018.4',
  '39615157.9',
  '76050616.1',
  '317194512.5',
  '234321242.3',
  '310371858.4',
  '4542664.9',
  '2279989.2',
  '6822654.1',
  '13847333.6',
  '165602386.9',
  '87884000',
  '28101536.1',
  '41948869.7',
  '31030688.2',
  '4618504.3',
  '120324.7',
  '35769517.2',
  '']]

##### 2. Número de países distintos en cada dataset. Coinciden?

Seleccionamos la columna que contiene el nombre de los países en ambos datasets.

In [9]:
paises_desarrollo = desarrollo_new.map(lambda x: (x[0]))
paises_deuda = deuda_new.map(lambda x: (x[0]))

Gracias a la función 'distinct' escogemos los valores únicos, y con 'count' los contamos.

In [10]:
paises_desarrollo.distinct().count()

51

In [11]:
paises_deuda.distinct().count()

123

A la vista de los resultados, el número de países distintos no coincide en las tablas. Mientras que en la de los indicadores de desarrollo son 51, en la de los indicadores de deuda el número asciende hasta 123.

##### 3. Total de deuda contraida por cada pais: total amount of debt (in USD) DT.AMT.MLAT.CD

Seleccionamos la columna donde se recoge el nombre del país y la columna donde se encuentra el indicador solicitado. Observamos los 10 primeros resultados.

In [12]:
deuda_contraida = deuda_new.map(lambda x: (x[0], x[5]))
deuda_contraida.take(10)

[('Afghanistan', '39107845'),
 ('Albania', '182197616.7'),
 ('Algeria', '0'),
 ('Angola', '124688691.9'),
 ('Armenia', '94331207.1'),
 ('Azerbaijan', '513623656.5'),
 ('Bangladesh', '1039564683'),
 ('Belarus', '595438826'),
 ('Belize', '29683326.5'),
 ('Benin', '91631639.5')]

##### 4. Media de los indicadores de deuda (tabla uno): DT.AMT.BLAT.CD, DT.DIS.BLAT.CD, DT.INT.BLAT.CD

Seleccionamos los indicadores que se solicitan en el enunciado, siguiendo el número de sus columnas. Vemos los resultados de una de ellas.

In [83]:
deuda_ind1 = deuda_new.map(lambda x: (x[2]))
deuda_ind2 = deuda_new.map(lambda x: (x[11]))
deuda_ind3 = deuda_new.map(lambda x: (x[18]))

In [84]:
deuda_ind1.take(5)

['61739336.9', '54250280.6', '95188724.6', '8473824016', '68968314.7']

Ahora seleccionamos aquellos que son dígitos, y los convertimos a tipo float al tener decimales incluidos.

In [85]:
deuda_ind1=deuda_ind1.filter(lambda x_int: x_int.isdigit())
deuda_ind1=deuda_ind1.map(lambda x: float(x))

deuda_ind2=deuda_ind2.filter(lambda x_int: x_int.isdigit())
deuda_ind2=deuda_ind2.map(lambda x: float(x))

deuda_ind3=deuda_ind3.filter(lambda x_int: x_int.isdigit())
deuda_ind3=deuda_ind3.map(lambda x: float(x))

Calculamos la media para cada uno de los indicadores, y se nos muestra el resultado:

In [86]:
deuda_ind1.mean(), deuda_ind2.mean(), deuda_ind3.mean()

(2890222062.565217, 4365716780.481482, 581647537.8)

##### 5. Los 20 paises con DT.AMT.DLXF.CD más alto

Para conseguirlos, se extrae el indicador solicitado en formato float, además de la columna del nombre del país. A continuación, se muestran los 20 datos más elevados de la primera de las dos columnas gracias a la función 'takeOrdered'.

In [17]:
deuda_alto = deuda_new.map(lambda x: (float(x[3]), x[0]))
deuda_alto.takeOrdered(20, key = lambda x: -x[0])

[(96218620836.0, 'China'),
 (90041840304.0, 'Brazil'),
 (66589761834.0, 'Russian Federation'),
 (51555031006.0, 'Turkey'),
 (48756295898.0, 'South Asia'),
 (31923507001.0, 'India'),
 (30916112654.0, 'Indonesia'),
 (27482093686.0, 'Kazakhstan'),
 (25218503927.0, 'Mexico'),
 (25197029299.0, 'Least developed countries: UN classification'),
 (20483289208.0, 'IDA only'),
 (14013783350.0, 'Romania'),
 (11985674439.0, 'Colombia'),
 (11067045628.0, 'Angola'),
 (10404814960.0, 'Cameroon'),
 (9506919670.0, 'Lebanon'),
 (9474257552.0, 'South Africa'),
 (8873505909.0, 'Vietnam'),
 (8336013891.0, 'Pakistan'),
 (8148995626.0, 'Ukraine')]

##### 6. Pais con los datos informados todos los años.

El cálculo se va a realizar sobre el dataset de indicadores de desarrollo, ya que es el que contiene datos de los países para diferentes años. Se selecciona la columna del país y añade un valor de 1 por cada año registrado. Después, un filtro para que si los 4 años están completos, se quede con ese país.

In [89]:
paises_completo = desarrollo_new.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y)
paises_completo = paises_completo.filter(lambda x: x[1] == 4)
paises_completo.take(5)

[('Malawi', 4)]

Como se puede observar, el único país que ha informado todos los años ha sido Malawi.

##### 7. Media anual de los distintos indicadores de desarrollo

En cada operación se ha procedido a filtrar por año, luego a seleccionar el dato en cuestión. Después se han filtrado aquellos datos que son dígitos, para luego convertirlos a tipo float. Por último, se ha calculado la media para cada indicador y año.

Medias anuales (2014, 2015, 2016) para la variable 'GC.DOD.TOTL.CN'.

In [19]:
desarrollo_2014 = desarrollo_new.filter(lambda x: x[2] == '2014 [YR2014]')
desarrollo_2014 = desarrollo_2014.map(lambda x: x[3])
desarrollo_2014=desarrollo_2014.filter(lambda x_int: x_int.isdigit())
desarrollo_2014=desarrollo_2014.map(lambda x: float(x))
desarrollo_2014.mean()

116610529707695.64

In [20]:
desarrollo_2015 = desarrollo_new.filter(lambda x: x[2] == '2015 [YR2015]')
desarrollo_2015 = desarrollo_2015.map(lambda x: x[3])
desarrollo_2015=desarrollo_2015.filter(lambda x_int: x_int.isdigit())
desarrollo_2015=desarrollo_2015.map(lambda x: float(x))
desarrollo_2015.mean()

184827281799554.75

In [21]:
desarrollo_2016 = desarrollo_new.filter(lambda x: x[2] == '2016 [YR2016]')
desarrollo_2016 = desarrollo_2016.map(lambda x: x[3])
desarrollo_2016=desarrollo_2016.filter(lambda x_int: x_int.isdigit())
desarrollo_2016=desarrollo_2016.map(lambda x: float(x))
desarrollo_2016.mean()

274875240137558.72

Medias anuales (2014, 2015, 2016) para la variable 'GC.DOD.TOTL.GD.ZS'.

In [22]:
desarrollo_2014 = desarrollo_new.filter(lambda x: x[2] == '2014 [YR2014]')
desarrollo_2014 = desarrollo_2014.map(lambda x: x[4])
desarrollo_2014=desarrollo_2014.map(lambda x: float(x))
desarrollo_2014.mean()

58.15214676864467

In [23]:
desarrollo_2015 = desarrollo_new.filter(lambda x: x[2] == '2015 [YR2015]')
desarrollo_2015 = desarrollo_2015.map(lambda x: x[4])
desarrollo_2015=desarrollo_2015.map(lambda x: float(x))
desarrollo_2015.mean()

59.76292583345264

In [24]:
desarrollo_2016 = desarrollo_new.filter(lambda x: x[2] == '2016 [YR2016]')
desarrollo_2016 = desarrollo_2016.map(lambda x: x[4])
desarrollo_2016=desarrollo_2016.map(lambda x: float(x))
desarrollo_2016.mean()

60.44188301621869

Medias anuales (2014, 2015, 2016) para la variable 'SP.DYN.CBRT.IN'.

In [25]:
desarrollo_2014 = desarrollo_new.filter(lambda x: x[2] == '2014 [YR2014]')
desarrollo_2014 = desarrollo_2014.map(lambda x: x[5])
desarrollo_2014=desarrollo_2014.map(lambda x: float(x))
desarrollo_2014.mean()

17.719702127659573

In [26]:
desarrollo_2015 = desarrollo_new.filter(lambda x: x[2] == '2015 [YR2015]')
desarrollo_2015 = desarrollo_2015.map(lambda x: x[5])
desarrollo_2015=desarrollo_2015.map(lambda x: float(x))
desarrollo_2015.mean()

16.904780487804878

In [27]:
desarrollo_2016 = desarrollo_new.filter(lambda x: x[2] == '2016 [YR2016]')
desarrollo_2016 = desarrollo_2016.map(lambda x: x[5])
desarrollo_2016=desarrollo_2016.map(lambda x: float(x))
desarrollo_2016.mean()

16.236916666666666

##### 8. Podrías decirme el total de deuda acumulada DT.AMT.MLAT.CD por los 10 países con un valor en media menor de SP.DYN.CBRT.IN

En primer lugar había que localizar los diez países con menor tasa de natalidad (SP.DYN.CBRT.IN), dato que se localiza en el dataset de desarrollo. Se ha seleccionado el año 2016 por ser el último del que se tienen datos de una cantidad grande de países. Después se han seleccionado la columna con los datos (en formato float) y la de los nombres del país. Por último, se han ordenado y mostrado los 10 con un indicador más pequeño. Esos países son los que cuentan con una tasa de natalidad más baja, por lo que serán los que se proceda a buscar en el dataset de deuda.

In [36]:
año_2016 = desarrollo_new.filter(lambda x: x[2] == '2016 [YR2016]')
natalidad_bajo = año_2016.map(lambda x: (float(x[5]), x[0]))
natalidad_bajo.takeOrdered(10, key = lambda x: x[0])

[(7.8, 'Japan'),
 (7.9, 'San Marino'),
 (8.8, 'Spain'),
 (9.4, 'Singapore'),
 (9.7, 'Hungary'),
 (10.3, 'Ukraine'),
 (10.445, 'Moldova'),
 (10.5, 'Switzerland'),
 (11.8, 'United Kingdom'),
 (12.0, 'Palau')]

Ahora procedemos a aplicar un filtro para buscar a cada uno de esos 10 países. Como se ha observado, de los 10 países tan solo Ucrania y Moldavia están en el dataset de deuda. Es por ello que solo podemos ofrecer estos dos resultados de los 10 países con tasa de natalidad más baja.

Lo que se ha hecho es buscar el país con el filtro y después seleccionar la columna del indicador que se pide y la del nombre del país.

In [125]:
ucrania = deuda_new.filter(lambda x: 'Ukraine' in x[0])
natalidad_deuda_ucrania = ucrania.map(lambda x: (float(x[5]), x[0]))
natalidad_deuda_ucrania.take(10)

[(1072472830.0, 'Ukraine')]

In [124]:
moldavia = deuda_new.filter(lambda x: 'Moldova' in x[0])
natalidad_deuda_moldavia = moldavia.map(lambda x: (float(x[5]), x[0]))
natalidad_deuda_moldavia.take(10)

[(135842865.5, 'Moldova')]

A modo de demostración, se puede observar que al buscar España en el dataset de deuda, no arroja ningún resultado. Así ha ocurrido con 8 de los 10 países, exceptuando Ucrania y Moldavia.

In [122]:
españa = deuda_new.filter(lambda x: 'Spain' in x[0])
natalidad_deuda_españa = españa.map(lambda x: (float(x[5]), x[0]))
natalidad_deuda_españa.take(10)

[]

In [123]:
japan = deuda_new.filter(lambda x: 'Japan' in x[0])
natalidad_deuda_japan = japan.map(lambda x: (float(x[5]), x[0]))
natalidad_deuda_japan.take(10)

[]

##### 9. ¿Hay alguna relación entre los nacimientos y el indicador DT.AMT.DLXF.CD? ¿Cómo podrías demostrarlo?

Lo he intentado pero no he conseguido sacarlo. Una opción que he barajado ha sido la de unir ambas tablas y luego hacer un scatterplot, o bien un cálculo de la correlación de ambas variables.